# Test pour discretiser l'espace d'état du jeu Flappy Bird

In [1]:
import numpy as np

In [1]:
# [float,float]

def arg_state(value,tab):
    bestIndex = 0
    index = 0
    while index < len(tab) and value > tab[index]:
        bestIndex = index
        index+=1
    return bestIndex
    
    
step = 30
min_distance = 0
max_distance = 2

tab = np.linspace(min_distance,max_distance,step)
print(tab)

NameError: name 'np' is not defined

In [2]:

xdif = 308
"""
if xdif < 300:
    xdif = int(xdif) - (int(xdif) % 5)
else:
    xdif = 300
print(xdif)
"""

ydif = -1.9904
"""
if ydif > 10 : 
    ydif = 10
elif ydif > 0 : 
    ydif = int(ydif) - (int(ydif) % 1)
elif ydif < -10 : 
    ydif = -10
elif ydif < 0 :
    ydif = int(ydif) + (int(ydif) % 1)
"""
print(ydif)

def get_state2(xdif,ydif,speed):
    hauteur_carte = 512
    largeur_carte = 288
    xdif *= largeur_carte
    ydif *= hauteur_carte
    if xdif < 300:
        xdif = int(xdif) - (int(xdif) % 5)
    else:
        xdif = 300
    if ydif > 10 : 
        ydif = 10
    elif ydif > 0 : 
        ydif = int(ydif) - (int(ydif) % 1)
    elif ydif < -10 : 
        ydif = -10
    elif ydif < 0 :
        ydif = int(ydif) + (int(ydif) % 1)
    return (xdif,ydif,speed)
        

    
    
states = [(j,i,z) for i in range(-10,11,1) for j in range(0,305,5) for z in range(-10,11)]
Q = dict()
actions = [0,1]
for state in states:
    for action in actions :
        Q[(state,action)] = 0
        
        
#print(Q)  
print(get_state2(xdif,ydif,8))
print(Q[(get_state2(xdif,ydif,8),1)])



-1.9904
(300, -10, 8)
0


In [3]:

hauteur_carte = 512
largeur_carte = 288

wvalue = 1.65 * largeur_carte
hvalue = -0.0117 * hauteur_carte

print(wvalue , hvalue)




475.2 -5.9904


In [4]:
value = 50
print(arg_state(value,tab))

NameError: name 'tab' is not defined

In [5]:
import gym
import flappy_bird_gym
import time
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

class MyFlappyEnv:
    """ Custom Flappy Env :
        * state : [horizontal delta of the next pipe, vertical delta, vertical velocity]
    """

    def __init__(self):
        self.env = flappy_bird_gym.make('FlappyBird-v0')
        self.env._normalize_obs = False
        self._last_score = 0
    def __getattr__(self,attr):
        return self.env.__getattribute__(attr)
    
    def step(self,action):
        obs, reward, done, info = self.env.step(action)
        if done:
            reward -=1000
        player_x = self.env._game.player_x
        player_y = self.env._game.player_y

        return np.hstack([obs,self.env._game.player_vel_y]),reward, done, info
    def reset(self):
        return np.hstack([self.env.reset(),self.env._game.player_vel_y])

def test_gym(fps=30):
    env = gym.make('Taxi-v3')
    env.reset()
    r = 0
    for i in range(100):
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        r += reward
        env.render()
        time.sleep(1/fps)
        print(f"iter {i} : action {action}, reward {reward}, state {type(obs)} ")
        if done:
            break
    print(f"reward cumulatif : {r} ")
 
def test_flappy(fps=30):
    env = flappy_bird_gym.make('FlappyBird-v0')
    env.reset()
    r = 0
    for i in range(100):
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        r += reward
        env.render()
        time.sleep(1/fps)
        print(f"iter {i} : action {action}, reward {reward}, state {obs} {info}, {env._game.player_vel_y}")
        print("data : ",env._game)
        if done:
            break
    print(f"reward cumulatif : {r} ")
 
def play_env(agent,max_ep=500,fps=-1,verbose=False):
    """
        Play an episode :
        * agent : agent with two functions : act(state) -> action, and store(state,action,state,reward)
        * max_ep : maximal length of the episode
        * fps : frame per second,not rendering if <=0
        * verbose : True/False print debug messages
        * return the cumulative reward
    """
    obs = agent.env.reset()
    cumr = 0
    for i in range(max_ep):
        last_obs = obs
        action = agent.act(obs)
        obs,reward,done,info = agent.env.step(int(action))
        agent.store(last_obs,action,obs,reward)
        cumr += reward
        if fps>0:
            agent.env.render()
            if verbose: print(f"iter {i} : {action}: {reward} -> {obs} ")        
            time.sleep(1/fps)
        if done:
            break
    return cumr

def transform_policy(env,policy) :
    # On transforme notre dictionnaire en Pi : State -> Int (indice action)
    new_policy = dict()
    for state in policy:
        actions = env.env.P[state]
        for action in actions :
            if actions[action] == policy[state]:
                new_policy[state] = action
    return new_policy

class AgentRandom:
    """
         A simple random agent
    """
    def __init__(self,env):
        self.env = env

    def act(self,obs):
        return self.env.action_space.sample()

    def store(self,obs,action,new_obs,reward):
        pass

class AgentPolicy:
    """
        Agent following a policy pi : pi is a dictionary state -> action
    """
    def __init__(self,env,pi):
        self.env = env
        self.pi = pi

    def act(self,obs):
        return self.pi[obs]

    def store(self,obs,action,new_obs,reward):
        pass

    def getPi(self) :
        return self.pi

    def getEnv(self) : 
        return self.env

    def setPi(self,pi):
        self.pi = pi


vitesse_min = -11
vitesse_max = 11
STEP = 50
        
def arg_state(value):
    start = time.time()
    value = value
    step = STEP
    min_distance = -1
    max_distance = 2.5
    tab = np.linspace(min_distance,max_distance,step)
    bestIndex = 0
    index = 0
    while index < len(tab) and value > tab[index]:
        bestIndex = index
        index+=1
    end = time.time()
    print("time : ",(end-start))
    return bestIndex


class FlappyAgent:
    """
        Agent following a policy pi : pi is a dictionary state -> action
    """
    def __init__(self,env,pi):
        self.env = env
        self.pi = pi

    def act(self,obs):
        horizontal_value,vertical_value = obs[0],obs[1]
        return self.pi[get_state2(horizontal_value,vertical_value,self.env._game.player_vel_y)]

    def store(self,obs,action,new_obs,reward):
        pass

    def getPi(self) :
        return self.pi

    def getEnv(self) : 
        return self.env

    def setPi(self,pi):
        self.pi = pi


import random
from tqdm import tqdm    

def argmaxQlearning(actions , state, Q) :
    if len(actions) < 1 :
        return None
    else :  
        best_action = actions[0]
        best_value = Q[(state,best_action)]
        for action in actions :
            value = Q[(state,action)]
            if value > best_value :
                best_value = value
                best_action = action
        return best_action

def maxQlearning(actions , state, Q) :
    if len(actions) < 1 :
        return None
    else :  
        best_action = actions[0]
        best_value = Q[(state,best_action)]
        for action in actions :
            value = Q[(state,action)]
            if value > best_value :
                best_value = value
        return best_value

def Qlearning(env, gamma = 0.9, eps=0.1, alpha=0.05,nbiter = 1000000) :
    step = STEP
    """1 - Initialisation of the Q-table """
    states = [(j,i,z) for i in range(-10,11,1) for j in range(0,305,5) for z in range(-10,11)]
    actions = [0,1]
    Q = dict()
    actions = [0,1]
    for state in states:
        for action in actions :
            Q[(state,action)] = 0
    
    
    print("Execution Q learning algorithm")
    for _ in tqdm(range(nbiter)) :
        env.reset()
        """ On initialise le premier état """
        horizontal_value,vertical_value = env._get_observation()[0],env._get_observation()[1]
        current_state = get_state2(horizontal_value,vertical_value,env._game.player_vel_y) # (arg_state(horizontal_value),arg_state(vertical_value),env._game.player_vel_y)
        final_state_reached = False
        """2 - Tant qu'on a pas atteint un état final"""
        iter = 0
        reward_t = 0
        while not final_state_reached :
            """2.1 - On doit choisir une action a faire """
            if random.uniform(0,1) < eps :
                """Exploration : on selectionne une action au hasard"""
                action = actions[random.randint(0, (len(actions)-1))]
            else :
                """Exploitation : on selectionne l'action avec la valeur maximale"""
                action = argmaxQlearning(actions,current_state,Q)
            """2.2 - Mise à jour de la table Q"""
            obs,reward,done,_ = env.step(int(action))
            horizontal_value,vertical_value = obs[0],obs[1]

            nextState = get_state2(horizontal_value,vertical_value,env._game.player_vel_y)
            
            """2.3 - Verification de l'etat final"""
            final_state_reached = done
            if final_state_reached :
                reward = -1000
            
            
            # print("vitesse : ",env._game.player_vel_y)
            
            
            # print((horizontal_value,vertical_value))
            # print('state : ',nextState,' reward : ',reward_t)
            # print((current_state,action))
            Q[(current_state,action)] = Q[(current_state,action)] + (alpha * (reward + ((gamma * maxQlearning(actions , nextState, Q)) - Q[(current_state,action)])))
            
            """2.4 - Mise à jour de l'état courant"""  
            current_state = nextState
            iter+=1
            if iter > 300 :
                print("yes")
        """3 - transformation de la table Q en politique""" 
        #print("etat final atteint en ",iter,"itérations")
    print(Q)
    PI = dict()
    for state in states:
        #print(state)
        bestAction = actions[0]
        bestValue = Q[(state,bestAction)]
        for action in actions : 
            value = Q[(state,action)]
            if value > bestValue :
                bestValue = value
                bestAction = action
        PI[state] = bestAction
    # print(PI)
    # print(Q.values())
    return PI

In [1]:
import numpy as np

In [2]:
liste = np.array([1,2,3])

In [3]:
liste.max()

3

In [6]:
A = {1 : list() , 2 : list()}

In [7]:
print(A)

{1: [], 2: []}


In [8]:
x = ["my", "unlimited", "sadness"]
for i in reversed(x):
    print(i)

sadness
unlimited
my
